## Скрипт построения графа, подсчета рейтинга по PageRank и визуализации

In [ ]:
#coding: utf-8
import networkx
from networkx.readwrite import json_graph

import json
import ast

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# формируем список пользователей из файла
def load_users(filename):
    users={}
    with open(filename,'r') as f:
        for line in f:
            user = ast.literal_eval(line.rstrip('\n'))
            users[user['id']]= user['friends']
    return users

In [ ]:
# строим граф
# node - ключи массива user, ноды - значения
def generate_graph(users):
    result = networkx.Graph()
    for user in users:
        #print user
        #print users[user]
        
        result.add_node(user)
        
        for friend in users[user]:
            if friend in users:
                result.add_edge(user, friend)
    return result

In [ ]:
# проверяем что содержится в файле
with open('users.txt','r') as f:
    for line in f:
        print line
        break

In [ ]:
users = load_users('users.txt')

In [ ]:
# генерим граф
g = generate_graph(users)

In [ ]:
# считаем pageRank
x = networkx.pagerank(g)

In [ ]:
# выводим список ID, отсортированных по PageRank
for i in sorted(x.keys(), key=lambda z: -x[z])[:10]:
    print i, x[i]

In [ ]:
# записываем массив с pagerank в файл
# эти данные будут учитываться при построении размеров узлов
with open('pagerank.txt', 'w') as f:
    f.write('id' + '\t' + 'pagerank' + '\n')
    for i in x.keys():
        f.write(str(i) + '\t' + str(x[i]) + '\n')

### Данные по ID: https://api.vk.com/method/users.get?user_id=12345

In [ ]:
data = json_graph.node_link_data(g)
with open('graph.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
%%html
<div id="d3-example" width='1000px'></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [ ]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.
    
    // First, we specify the size of the canvas containing
    // the visualization (size of the <div> element).
    var width = 800,
        height = 600;

    // We create a color scale.
    var color = d3.scale.category10();

    // We create a force-directed dynamic graph layout.
    var force = d3.layout.force()
        .charge(-200)
        .linkDistance(100)
        .size([width, height])
        .friction(.2);

    // In the <div> element, we create a <svg> graphic
    // that will contain our interactive visualization.
    var svg = d3.select("#d3-example").select("svg")
    if (svg.empty()) {
        svg = d3.select("#d3-example").append("svg")
                    .attr("width", width)
                    .attr("height", height);
    }
    
    
    d3.tsv('pagerank.txt', function(data) {
        var prdata = {};
        data.forEach(function(d) {
            prdata[d.id] = d.pagerank;
        });
    
        // We load the JSON file.
        d3.json("graph.json", function(graph) {
            // In this block, the file has been loaded
            // and the 'graph' object contains our graph.
            
            graph.fixed = true;
            graph.x = width;
            graph.y = height / 2 - 80;
            
            // We load the nodes and links in the force-directed
            // graph.
            force.nodes(graph.nodes)
                .links(graph.links)
                .start();

            // We create a <line> SVG element for each link
            // in the graph.
            var link = svg.selectAll(".link")
                .data(graph.links)
                .enter().append("line")
                .attr("class", "link");

            // We create a <circle> SVG element for each node
            // in the graph, and we specify a few attributes.
            var node = svg.selectAll(".node")
                .data(graph.nodes)
                .enter().append("circle")
                .attr("class", "node")
                //.attr("r", 5)  // radius
                .attr("r", function(d) { return prdata[d.id] * 2000; })  // radius
                .style("fill", function(d) {
                    // The node color depends on the club.
                    return color(d.club); 
                })
                .call(force.drag);
                       
            // The name of each node is the node number.
            node.append("title")
                .text(function(d) { return d.id; });
            
            // We bind the positions of the SVG elements
            // to the positions of the dynamic force-directed graph,
            // at each time step.
            force.on("tick", function() {
                link.attr("x1", function(d) { return d.source.x; })
                    .attr("y1", function(d) { return d.source.y; })
                    .attr("x2", function(d) { return d.target.x; })
                    .attr("y2", function(d) { return d.target.y; });

                node.attr("cx", function(d) { return d.x; })
                    .attr("cy", function(d) { return d.y; });
            });
        });
    });
});